##  Feature Engineering – CMAPSS Dataset

In [1]:
# 0.1 Imports and Data Loading
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pathlib import Path

# Set column names
col_names = [
    "engine_id", "cycle", "op_setting_1", "op_setting_2", "op_setting_3"
]
col_names.extend([f"sensor_{i}" for i in range(1, 22)])

# Directory with the Train files
# Use a relative path to go up 3 folders to the project root, then down into data
# Fallback path if the one above fails
data_dir = Path('data')

# Load all four files and add an identifier column
datasets = {}
for fd_id in range(1, 5):
    file_path = data_dir / 'raw' / f'train_{fd_id}.txt'
    
    datasets[f"FD00{fd_id}"] = pd.read_csv(
        file_path, sep=' ', header=None, names=col_names
    )
    
    datasets[f"FD00{fd_id}"]["dataset_id"] = f"FD00{fd_id}"

# Merge into a single DataFrame
df = pd.concat(datasets.values(), ignore_index=True)

print(f"Shape of the merged DataFrame: {df.shape}")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data\\raw\\train_1.txt'

#### Aggregate Features – Mean, Std, Min, Max per Engine

This code snippet dynamically selects all sensor columns in the C-MAPSS dataset and calculates engine-wise aggregate statistics such as mean, standard deviation, minimum, and maximum for each sensor across the lifecycle of each engine. By grouping data based on engine_id, it summarizes the behavior of each sensor over all operational cycles for that engine. The resulting aggregated features condense time-series sensor data into static descriptors, which help capture overall degradation patterns and operational characteristics unique to each engine. Flattening the multi-level column index makes subsequent analysis easier by giving intuitive names measuring each sensor's aggregate behavior.

This aggregation step is crucial for the C-MAPSS dataset because it transforms noisy and complex time-series sensor measurements into a more compact and informative format that summarizes the full history of each engine. These static features are essential inputs for machine learning models predicting Remaining Useful Life (RUL) or potential faults. They improve model performance by reducing dimensionality, increasing interpretability, and enhancing the ability to capture long-term trends and degradation characteristics, thus enabling more reliable predictive maintenance and health monitoring of aircraft engines.

In [ ]:
# Sensor columns
sensor_cols = [col for col in df.columns if 'sensor_' in col]

# Engine-wise aggregrate features (static for each engine)
engine_aggs = df.groupby('engine_id')[sensor_cols].agg(['mean', 'std', 'min', 'max'])
engine_aggs.columns = ['_'.join(col) for col in engine_aggs.columns]
engine_aggs.reset_index(inplace=True)
print(f"Aggregate feature matrix shape: {engine_aggs.shape}")
engine_aggs.tail()


Aggregate feature matrix shape: (260, 85)


,engine_id,sensor_1_mean,sensor_1_std,sensor_1_min,sensor_1_max,sensor_2_mean,sensor_2_std,sensor_2_min,sensor_2_max,sensor_3_mean,...,sensor_19_min,sensor_19_max,sensor_20_mean,sensor_20_std,sensor_20_min,sensor_20_max,sensor_21_mean,sensor_21_std,sensor_21_min,sensor_21_max
255,256,475.140000,27.486466,445.0,518.67,583.608466,38.064278,536.09,644.12,1431.573988,...,84.93,100.0,21.881902,10.297101,10.26,39.14,13.137912,6.167984,6.2293,23.4157
256,257,473.989741,26.798684,445.0,518.67,582.076214,37.081296,536.12,643.39,1427.211618,...,84.93,100.0,21.375210,10.022302,10.38,39.17,12.826693,6.010580,6.2410,23.4548
257,258,475.573636,27.340533,445.0,518.67,583.308322,38.870335,536.00,644.05,1429.926853,...,84.93,100.0,21.690490,10.152721,10.34,38.98,13.010299,6.094522,6.2127,23.3676
258,259,472.357659,25.644641,445.0,518.67,579.046976,36.139349,535.78,643.15,1418.357512,...,84.93,100.0,20.772488,9.668143,10.30,39.02,12.472388,5.807734,6.2383,23.5015
259,260,473.033608,26.415225,445.0,518.67,579.254620,37.604858,536.13,644.15,1417.488165,...,84.93,100.0,20.835095,9.955657,10.38,39.18,12.498821,5.973274,6.1757,23.5412


#### Rolling Statistics and Trends

This code calculates rolling mean and rolling standard deviation for each sensor reading within each engine across cycles, effectively capturing short-term temporal patterns. By grouping data per engine_id and computing these statistics over a moving window of 5 cycles, the code smooths sensor readings and quantifies local variability, even at the start of the series via min_periods=1. These additional rolling features are appended with descriptive column names, providing a richer temporal context beyond raw sensor values.

This process is vital for the C-MAPSS dataset because it transforms raw noisy sensor signals into features that reflect recent dynamic changes in engine condition. The rolling mean highlights underlying trends by reducing noise, while the rolling std signals fluctuations or irregularities that may indicate emerging faults. Together with static aggregate features, these rolling statistics enable machine learning models to better understand engine degradation over multiple time scales, leading to more accurate Remaining Useful Life (RUL) predictions and improved preventive maintenance strategies.

In [ ]:
# Add rolling means (windows=5 cycles) and rolling std for each sensor per engine
for col in sensor_cols:
  df[f"{col}_rollmean5"] = df.groupby('engine_id')[col].rolling(window=5, min_periods=1).mean().reset_index(level=0, drop=True)
  df[f"{col}_rollstd5"] = df.groupby('engine_id')[col].rolling(window=5, min_periods=1).std().reset_index(level=0, drop=True)

# Prepare list of columns to display
cols_to_show = sensor_cols + [f"{col}_rollmean5" for col in sensor_cols] + [f"{col}_rollstd5" for col in sensor_cols]

# Display first 10 rows of the selected columns
df[cols_to_show].head(10)

,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,sensor_8,sensor_9,sensor_10,...,sensor_12_rollstd5,sensor_13_rollstd5,sensor_14_rollstd5,sensor_15_rollstd5,sensor_16_rollstd5,sensor_17_rollstd5,sensor_18_rollstd5,sensor_19_rollstd5,sensor_20_rollstd5,sensor_21_rollstd5
0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,...,0.438406,0.035355,5.041671,0.008697,0.0,0.000000,0.0,0.0,0.042426,0.003253
2,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,...,0.404475,0.026458,3.717450,0.007640,0.0,1.154701,0.0,0.0,0.055076,0.044573
3,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,...,0.495950,0.029439,3.050906,0.028117,0.0,1.000000,0.0,0.0,0.076322,0.037977
4,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,...,0.432574,0.025884,2.651326,0.025953,0.0,1.095445,0.0,0.0,0.073621,0.033498
5,518.67,642.10,1584.47,1398.37,14.62,21.61,554.67,2388.02,9049.68,1.3,...,0.425417,0.023452,0.958697,0.025727,0.0,1.140175,0.0,0.0,0.051186,0.031436
6,518.67,642.48,1592.32,1397.77,14.62,21.61,554.34,2388.02,9059.13,1.3,...,0.425652,0.021679,0.643141,0.023476,0.0,1.140175,0.0,0.0,0.086718,0.021634
7,518.67,642.56,1582.96,1400.97,14.62,21.61,553.85,2388.00,9040.80,1.3,...,0.429919,0.021679,1.149274,0.022477,0.0,0.836660,0.0,0.0,0.086487,0.034405
8,518.67,642.12,1590.98,1394.80,14.62,21.61,553.69,2388.05,9046.46,1.3,...,0.341101,0.008944,3.205438,0.020740,0.0,0.836660,0.0,0.0,0.077136,0.038939
9,518.67,641.71,1591.24,1400.46,14.62,21.61,553.59,2388.05,9051.70,1.3,...,0.358260,0.014142,2.883881,0.020493,0.0,0.836660,0.0,0.0,0.062849,0.058103


In [ ]:
# Check for missing values in percentage
missing_values = df.isnull().sum() / df.shape[0] * 100
missing_values[missing_values > 0].sort_values(ascending=False)


,0
sensor_1_rollstd5,0.162136
sensor_2_rollstd5,0.162136
sensor_3_rollstd5,0.162136
sensor_4_rollstd5,0.162136
sensor_5_rollstd5,0.162136
sensor_6_rollstd5,0.162136
sensor_7_rollstd5,0.162136
sensor_8_rollstd5,0.162136
sensor_9_rollstd5,0.162136
sensor_10_rollstd5,0.162136


In [ ]:
# drop the rows with missing values
df.dropna(inplace=True)

#### Sensor Value Normalization

In [ ]:
# Normalize all sensor columns and rolling feature columns (per sensor globally for simplicity)
features_to_scale = [col for col in df.columns if ('sensor_'in col) or ('roll' in col)]

# StandardScaler (mean=0, std=1)
scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[features_to_scale] = scaler.fit_transform(df[features_to_scale])

# Confirm scaled feature distribution
df_scaled[features_to_scale].describe().T[['mean', 'std']]

,mean,std
sensor_1,-4.544661e-16,1.000003
sensor_2,-8.123582e-16,1.000003
sensor_3,2.499564e-16,1.000003
sensor_4,-1.364819e-15,1.000003
sensor_5,-3.749346e-16,1.000003
...,...,...
sensor_19_rollstd5,-3.692537e-17,1.000003
sensor_20_rollmean5,5.908060e-16,1.000003
sensor_20_rollstd5,1.732652e-16,1.000003
sensor_21_rollmean5,-1.255463e-15,1.000003


#### Feature Matrix Construction & Validation

In [ ]:
# Select all numeric columns except for engine_id and cycle as feature matrix
exclude_cols = ['engine_id', 'cycle']
feature_cols = [col for col in df_scaled.columns if col not in exclude_cols]

# Check for missing values
print("Missing values per features columns:")
print(df_scaled[feature_cols].isnull().sum())

Missing values per features columns:
op_setting_1           0
op_setting_2           0
op_setting_3           0
sensor_1               0
sensor_2               0
                      ..
sensor_19_rollstd5     0
sensor_20_rollmean5    0
sensor_20_rollstd5     0
sensor_21_rollmean5    0
sensor_21_rollstd5     0
Length: 67, dtype: int64


In [ ]:
# Final feature matrix
X = df_scaled[feature_cols]
print(f"Final feature matrix shape: {X.shape}")
X.head()

Final feature matrix shape: (160099, 67)


,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,sensor_6,sensor_7,...,sensor_17_rollmean5,sensor_17_rollstd5,sensor_18_rollmean5,sensor_18_rollstd5,sensor_19_rollmean5,sensor_19_rollstd5,sensor_20_rollmean5,sensor_20_rollstd5,sensor_21_rollmean5,sensor_21_rollstd5
1,0.0019,-0.0003,100.0,1.079459,1.054654,1.056148,1.043391,1.108018,1.115325,1.114508,...,1.374561,-1.455685,1.262047,-1.184951,0.705019,-0.802578,1.461675,-1.367305,1.462277,-1.374635
2,-0.0043,0.0003,100.0,1.079459,1.059362,1.023736,1.051169,1.108018,1.115325,1.117437,...,1.345246,-1.369314,1.262047,-1.184951,0.705019,-0.802578,1.458692,-1.364754,1.457485,-1.360744
3,0.0007,0.0000,100.0,1.079459,1.059362,0.979730,1.034073,1.108018,1.115325,1.118528,...,1.352574,-1.380886,1.262047,-1.184951,0.705019,-0.802578,1.455242,-1.360468,1.456473,-1.362962
4,-0.0019,-0.0002,100.0,1.079459,1.059833,0.980238,1.065990,1.108018,1.115325,1.115944,...,1.365766,-1.373746,1.262047,-1.184951,0.705019,-0.802578,1.453620,-1.361012,1.457004,-1.364467
5,-0.0043,-0.0001,100.0,1.079459,1.053477,0.993947,1.008393,1.108018,1.115325,1.119791,...,1.356972,-1.370401,1.262047,-1.184951,0.705019,-0.802578,1.451830,-1.365538,1.455061,-1.365160


In [ ]:
X.info()
# X.drop(columns=['dataset_id'],inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 160099 entries, 1 to 160358
Data columns (total 66 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   op_setting_1         160099 non-null  float64
 1   op_setting_2         160099 non-null  float64
 2   op_setting_3         160099 non-null  float64
 3   sensor_1             160099 non-null  float64
 4   sensor_2             160099 non-null  float64
 5   sensor_3             160099 non-null  float64
 6   sensor_4             160099 non-null  float64
 7   sensor_5             160099 non-null  float64
 8   sensor_6             160099 non-null  float64
 9   sensor_7             160099 non-null  float64
 10  sensor_8             160099 non-null  float64
 11  sensor_9             160099 non-null  float64
 12  sensor_10            160099 non-null  float64
 13  sensor_11            160099 non-null  float64
 14  sensor_12            160099 non-null  float64
 15  sensor_13            1

In [ ]:
# save the processed feature matrix to a CSV file
output_path = data_dir / "processed_feature_matrix.csv"
X.to_csv(output_path, index=False)

#### Feature Engineering Summary

- Features Created:
  - Aggregate statistics per engine: mean, std, min, max for each sensor (static features)
  - Rolling-window features (window=5): rolling mean and std trend for each sensor per cycle/engine (dynamic features)
  - All raw, aggregate, and rolling features scaled using StandardScaler (zero mean, unit variance)

- Validation:
  - Verified presence of missing values in the final feature set.
  - Final feature matrix contains [INSERT VALUE: X.shape] engineered features per cycle.

- ### Observations:


* This file starts by loading and combining all four NASA engine datasets, just like the Day 5 file did.
* The main job is "Feature Engineering," which just means **creating new, smarter columns** to help the computer find patterns.
* First, it creates "summary" columns by calculating the average, min, and max for each sensor over each engine's *entire life*.
* Second, it creates "rolling" columns by using a **sliding average** (of the last 5 cycles) for every sensor, which helps to smooth out noisy data.
* These new rolling columns created a few blank spots, so the code's simple fix was to **delete any row** that had a blank.
* After making the new columns, it "scales" all the sensor data, which just makes all the numbers "fair" by putting them on the same scale.
* Finally, the code saves this new, super-detailed table (with 66 columns) to a new file named **`processed_feature_matrix.csv`**.